In [ ]:
import icepyx as ipx
import numpy as np                   # Numeric Python
import matplotlib.pyplot as plt      # Plotting routines
import h5py                          # general HDF5 reading/writing library
import rioxarray as rx               # Package to read raster data from hdf5 files
from pyproj import Transformer, CRS  # libraries to allow coordinate transforms
import glob                          # Package to locate files on disk
import os                            # File-level utilities
import re                            # regular expressions for string interpretation
import xarray as xr
import dask
# %matplotlib inline

In [ ]:
%matplotlib widget 
%config InlineBackend.figure_format = 'retina'

# construct query

In [ ]:
short_name = 'ATL03'
spatial_extent = [-101.8,-75.23,-100.50,-74.80]
date_range = ['2019-01-01','2019-01-30']

In [ ]:
region_a = ipx.Query(short_name, spatial_extent, date_range)

In [ ]:
print(region_a.product)
print(region_a.dates)
print(region_a.start_time)
print(region_a.end_time)
print(region_a.cycles)
print(region_a.tracks)
print(region_a.product_version)
region_a.visualize_spatial_extent()

In [ ]:
region_a.product_summary_info()
print(region_a.latest_version())

In [ ]:
region_a.CMRparams

In [ ]:
region_a.avail_granules()

In [ ]:
region_a.avail_granules(ids=True)

In [ ]:
region_a.subsetparams()

# get data

In [ ]:
earthdata_uid = 'loicbachelot'
email = 'loic.bachelot@gmail.com'
region_a.earthdata_login(earthdata_uid, email)

In [ ]:
region_a.order_granules()

In [ ]:
region_a.granules.orderIDs

In [ ]:
path = '/tmp/product_comp_ATL03/'
region_a.download_granules(path)

# read data and plot

## quick look at the data opening one beam data

In [ ]:
! h5ls /tmp/product_comp_ATL03/processed_ATL03_20190115051807_02710210_005_01.h5/gt2r/heights

In [ ]:
ds = xr.open_dataset(
        filename_or_obj="/tmp/product_comp_ATL03/processed_ATL03_20190115051807_02710210_005_01.h5",
        group=f"gt2l/heights",# take the middle pair track out of pt1, pt2 & pt3
    )
ds

## Read a full file

In [ ]:
def ATL03_to_xarray(path):
    dataset_list = []
    for track in ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']:
        ds = xr.open_dataset(
            filename_or_obj=path,
            group=f"{track}/heights",
        )
        ds['gt_id'] = [track]
        ds['delta_time'] = np.arange(len(ds.delta_time.data))
        ds = ds.rename({'delta_time': 'id_point'})
        dataset_list.append(ds)
    dataset = xr.concat(dataset_list, dim = 'gt_id')
    return dataset

In [ ]:
ds = ATL03_to_xarray("/tmp/product_comp_ATL03/processed_ATL03_20190115051807_02710210_005_01.h5")
ds

# selecting one track and simple plotting

In [ ]:
gt2l = ds.sel(gt_id='gt2l')
gt2l = gt2l.where(gt2l.quality_ph == 0, drop=True)
gt2l

In [ ]:
plt.scatter(x=gt2l.lat_ph.data, y=gt2l.h_ph.data)